Instalando bibliotecas necessárias 

In [ ]:
pip install jupyter


In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager


In [ ]:
pip install xlsxwriter

In [ ]:
pip install openpyxl

In [ ]:
pip install --upgrade pip  pandas as pd

Iniciando extração

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
import shutil
import time



C:\Users\anaph\AppData\Local\Temp\ipykernel_6984\4020833464.py:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Inicializar o driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Maximizar a janela do navegador
driver.maximize_window()

# Abrir o site
driver.get("https://sisdagro.inmet.gov.br/sisdagro/app/monitoramento/bhs")

# Recarregar a página para garantir o carregamento correto
driver.refresh()

# Lista para armazenar os dados capturados de cada estação
dados_estacoes = []

# Função para capturar os dados da tabela
def capturar_dados_tabela():
    tabela_element = driver.find_element(By.ID, "gridDiv")
    linhas = tabela_element.find_elements(By.TAG_NAME, "tr")  # Capturar todas as linhas da tabela
    dados_tabela = []

    for linha in linhas:
        colunas = linha.find_elements(By.TAG_NAME, "td")  # Capturar todas as colunas de cada linha
        dados_linha = [coluna.text for coluna in colunas]  # Extrair o texto de cada coluna
        if dados_linha:  # Evitar adicionar linhas vazias
            dados_tabela.append(dados_linha)
    
    return dados_tabela

# Esperar até que o JavaScript esteja completamente carregado
try:
  
    WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, "ext-gen43"))
    )
    print("Página carregada completamente.")

    # Esperar o campo de data inicial estar visível
    data_inicial = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, "data-inicial"))
    )
    # Limpar o campo de data e inserir a nova data
    data_inicial.clear()
    data_inicial.send_keys("01/01/2018")
    data_inicial.send_keys(Keys.RETURN)

    # Esperar o campo de data final estar visível
    data_final = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, "data-final"))
    )
    # Limpar o campo de data e inserir a nova data
    data_final.clear()
    data_final.send_keys("31/08/2024")
    data_final.send_keys(Keys.RETURN)

    # Localizar o campo de texto "combo-estacoes" e clicar nele para abrir o dropdown
    combo_estacoes = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, "ext-gen43"))
    )
    combo_estacoes.click()  # Clica no campo para abrir o menu dropdown

    # Esperar que as opções apareçam (substituir o seletor conforme necessário)
    estacoes_options = WebDriverWait(driver, 1).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 'x-combo-list-item')]"))
    )

    # Percorrer e selecionar cada opção
    for option in estacoes_options:
        # Clicar em cada opção
        option.click()
        print(f"Selecionada estação: {option.text}")

        # Clicar no botão "Rodar" (ext-gen22) após a seleção
        botao_rodar = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.ID, "ext-gen22"))
        )
        botao_rodar.click()
        time.sleep(1)

        # Esperar o desaparecimento do elemento de carregamento
        WebDriverWait(driver, 1).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".classe-carregamento"))  # Substitua pelo identificador correto
        )
        print("Carregamento concluído.")
        
        # Capturar os valores dos IDs especificados
        estacao_resumo = driver.find_element(By.ID, "estacoesResumo").text
        solo_resumo = driver.find_element(By.ID, "culturaResumo").text
        periodo_inicio = driver.find_element(By.ID, "dataInicio").text
        periodo_fim = driver.find_element(By.ID, "dataFinal").text
        
        # Capturar os dados da tabela na tela (gridDiv)
        tabela_dados = capturar_dados_tabela()

        # Adicionar os dados capturados à lista de dados, incluindo a tabela
        dados_estacoes.append({
            "estacao": estacao_resumo,
            "solo": solo_resumo,
            "periodo_inicio": periodo_inicio,
            "periodo_fim": periodo_fim,
            "tabela_dados": tabela_dados
        })

        print(f"Dados capturados: Estação: {estacao_resumo}, Solo: {solo_resumo}, Início: {periodo_inicio}, Fim: {periodo_fim}")
        print(f"Dados da tabela: {tabela_dados}")

        # Clicar novamente no combo-estacoes para reabrir o dropdown, se necessário
        combo_estacoes.click()
        print("Botão Rodar clicado.")
     
except Exception as e:
    print(f"Erro ao interagir com a página: {e}")

finally:
    driver.quit()

# Exibir os dados capturados para verificação
for dado in dados_estacoes:
    print("terminou Coleta")


In [ ]:

# Lista para armazenar os dados tabulares finais
tabela_completa = []
colunas_definidas = False
colunas = []  # Para armazenar os cabeçalhos corretos

# Converter cada entrada de 'dados_estacoes' em formato tabular
for dado in dados_estacoes:
    estacao = dado["estacao"]
    solo = dado["solo"]
    periodo_inicio = dado["periodo_inicio"]
    periodo_fim = dado["periodo_fim"]
    tabela_dados = dado["tabela_dados"]

    # Usar a primeira linha da tabela como cabeçalho
    if not colunas_definidas:
        colunas = ["Estação", "Solo", "Período Início", "Período Fim"] + tabela_dados[0]  # Cabeçalhos reais
        colunas_definidas = True
    
    # Remover a primeira linha, que agora é o cabeçalho
    tabela_dados = tabela_dados[1:]
    
    # Para cada linha da tabela extraída, adicionamos as informações da estação, solo, e períodos
    for linha in tabela_dados:
        tabela_completa.append([estacao, solo, periodo_inicio, periodo_fim] + linha)

# Converter a lista completa em um DataFrame do pandas
df = pd.DataFrame(tabela_completa, columns=colunas)

# Exibir a tabela
print(df)

# Caminho para salvar os arquivos
caminho_base = "C:/Users/anaph/OneDrive/Área de Trabalho/DadosMeteorologicos/"

# Salvar a tabela em um arquivo Excel (usando openpyxl ou xlsxwriter)
df.to_excel(caminho_base + "dados_metereologicos.xlsx", index=False, engine='openpyxl')


                      Estação                        Solo Período Início  \
0           ABROLHOS (A) - BA  5 - Muito Arenoso (CAD 25)     01/01/2018   
1           ABROLHOS (A) - BA  5 - Muito Arenoso (CAD 25)     01/01/2018   
2           ABROLHOS (A) - BA  5 - Muito Arenoso (CAD 25)     01/01/2018   
3           ABROLHOS (A) - BA  5 - Muito Arenoso (CAD 25)     01/01/2018   
4           ABROLHOS (A) - BA  5 - Muito Arenoso (CAD 25)     01/01/2018   
...                       ...                         ...            ...   
186321  PASSA QUATRO (C) - MG      2 - Argiloso (CAD 100)     01/01/2018   
186322  PASSA QUATRO (C) - MG      2 - Argiloso (CAD 100)     01/01/2018   
186323  PASSA QUATRO (C) - MG      2 - Argiloso (CAD 100)     01/01/2018   
186324  PASSA QUATRO (C) - MG      2 - Argiloso (CAD 100)     01/01/2018   
186325  PASSA QUATRO (C) - MG      2 - Argiloso (CAD 100)     01/01/2018   

       Período Fim        Data    ARM    ALT   ETR   DEF    EXC   ETo      P  \
0      

In [24]:
#Incluir COLUNA UF
import pandas as pd


# Caminho do arquivo Excel
caminho_arquivo = r"C:/Users/anaph/OneDrive/Área de Trabalho/DadosMeteorologicos/dados_metereologicos.xlsx"

# Caminho do arquivo Excel
caminho_arquivo_saida = r"C:/Users/anaph/OneDrive/Área de Trabalho/DadosMeteorologicos/dados_metereologicos.xlsx"


# Leitura do arquivo Excel
df = pd.read_excel(caminho_arquivo)

# Função para extrair a UF e atualizar o nome da estação
def separar_uf(nome_estacao: str) -> tuple:
    """
    Extrai a UF do nome da estação e retorna o nome da estação sem a UF.
      """
    if pd.isna(nome_estacao) or not isinstance(nome_estacao, str):
        return nome_estacao, ""  # Retorna o valor original e uma string vazia para UF

    # Extrai a UF (últimos 2 caracteres após o hífen)
    if " - " in nome_estacao:
        nome, uf = nome_estacao.rsplit(" - ", 1)
        return nome.strip(), uf.strip()
    else:
        return nome_estacao, ""  # Caso não haja UF, retorna o nome original e uma string vazia

# Aplicar a função para criar as colunas "Estação" e "UF"
df[['Estação', 'UF']] = df['Estação'].apply(separar_uf).apply(pd.Series)

# Salvar o arquivo atualizado
df.to_excel(caminho_arquivo_saida, index=False)

print("Arquivo atualizado com sucesso!")



Arquivo atualizado com sucesso!


In [ ]:
pip install --upgrade pip unidecode

In [1]:
import pandas as pd
import httpx
from unidecode import unidecode
import logging

# Configuração do logger (apenas logs críticos e erros)
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# Dicionário de IDs não mapeados
ids_nao_mapeados = {
    "santana do livramento": 430016,
    "são félix do araguaia": 5107859,
    "são luiz do paraitinga": 3550001
}

# Dicionário de associações de nomes de municípios
locais_associacoes = {
    "abrolhos": "caravelas",
    "aguas emendadas": "planaltina",
    "alto da boa vista": "rio de janeiro",
    "arco verde": "recife",
    "arembepe": "camaçari",
    "avelar": "paty do alferes",
    "bom jesus do piaui": "bom jesus",
    "calcanhar": "touros",
    "caldeirao": "picos",
    "camaratuba": "mamanguape",
    "campo novo dos parecis": "campo novo do parecis",
    "campos": "campos dos goytacazes",
    "c. do mato dentro": "conceição do mato dentro",
    "ceara mirim": "ceará-mirim",
    "delfino": "vitória da conquista",
    "ecologia agrícola": "seropédica",
    "faculdade da terra de brasília": "brasília",
    "florianópolis-são jose": "são josé",
    "forte de copacabana": "rio de janeiro",
    "gleba celeste": "sorriso",
    "iauaretê": "são gabriel da cachoeira",
    "ilha de santana": "são luís",
    "ilha do mel": "paranaguá",
    "jacarepagua": "rio de janeiro",
    "luiz eduardo magalhaes": "luís eduardo magalhães",
    "mal. candido rondon": "marechal cândido rondon",
    "maria de fé": "maria da fé",
    "mocambinho": "pirapora",
    "moela": "itapira",
    "monte verde": "camanducaia",
    "morro dos cavalos": "são joão da serra",
    "nhumirim": "corumbá",
    "nova xav.": "nova xavantina",
    "padre ricardo remetter": "campo verde",
    "palmeira da missões": "palmeira das missões",
    "paratí": "paraty",
    "parque estadual chandless": "manoel urbano",
    "pico do couto": "petrópolis",
    "poxoreo": "poxoréu",
    "preguiças": "barreirinhas",
    "pres. kennedy": "presidente kennedy",
    "realengo": "rio de janeiro",
    "rio de janeiro-marambaia": "rio de janeiro",
    "rio urubu": "itacoatiara",
    "santa marta": "laguna",
    "santana do livramento": "santana do livramento",
    "são felix do araguaia": "são félix do araguaia",
    "são luis do paraitinga": "são luís do paraitinga",
    "sao paulo - mirante": "são paulo",
    "sao s.do paraiso": "são sebastião do paraíso",
    "serido": "caicó",
    "serra dos carajás": "parauapebas",
    "s.g.da cachoeira": "são gabriel da cachoeira",
    "s.j. do rio claro": "são josé do rio claro",
    "sta. r. de cassia": "santa rita de cássia",
    "tartarugualzinho": "tartarugalzinho",
    "tomé açu": "tomé-açu",
    "três maria": "três marias",
    "usina junqueira": "cravinhos",
    "vale do gurgueia": "cristino castro",
    "vila militar": "rio de janeiro",
    "xerem": "duque de caxias"
}

# Dicionário de mapeamento de UF (abreviação para nome completo)
uf_abreviatura_para_nome = {
    "AC": "Acre",
    "AL": "Alagoas",
    "AP": "Amapá",
    "AM": "Amazonas",
    "BA": "Bahia",
    "CE": "Ceará",
    "DF": "Distrito Federal",
    "ES": "Espírito Santo",
    "GO": "Goiás",
    "MA": "Maranhão",
    "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais",
    "PA": "Pará",
    "PB": "Paraíba",
    "PR": "Paraná",
    "PE": "Pernambuco",
    "PI": "Piauí",
    "RJ": "Rio de Janeiro",
    "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul",
    "RO": "Rondônia",
    "RR": "Roraima",
    "SC": "Santa Catarina",
    "SP": "São Paulo",
    "SE": "Sergipe",
    "TO": "Tocantins"
}

def preparar_nome_municipio(nome_municipio: str) -> str:
    """
    Prepara o nome do município para comparação:
    - Remove partes irrelevantes (após hífen ou parêntese)
    - Remove acentos
    - Converte para minúsculas
    - Remove espaços extras
    - Aplica associações definidas em locais_associacoes
    """
    if pd.isna(nome_municipio) or not isinstance(nome_municipio, str):
        return ""
    
    nome_limpo = nome_municipio.split("-")[0].split("(")[0]
    nome_normalizado = " ".join(unidecode(nome_limpo).lower().split())
    
    # Aplica a associação se existir
    if nome_normalizado in locais_associacoes:
        nome_normalizado = locais_associacoes[nome_normalizado]
    
    return nome_normalizado

def obter_dados_ibge() -> list:
    """
    Faz uma única requisição à API do IBGE e retorna uma lista com os dados dos municípios.
    """
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios"
    try:
        response = httpx.get(url, timeout=10)
        response.raise_for_status()
        return response.json()
    except httpx.HTTPStatusError as e:
        logger.error(f"Erro HTTP {e.response.status_code} ao fazer requisição para {url}")
    except httpx.RequestError as e:
        logger.error(f"Erro na requisição: {e}")
    return []

def criar_mapeamento_ibge(dados_ibge: list) -> dict:
    """
    Cria um dicionário com chave (nome_normalizado, UF_full) e valor código IBGE,
    para buscas rápidas.
    """
    mapeamento = {}
    for municipio in dados_ibge:
        nome = preparar_nome_municipio(municipio['nome'])
        uf_api = municipio['microrregiao']['mesorregiao']['UF']['sigla']
        uf_full = uf_abreviatura_para_nome.get(uf_api.upper(), uf_api).lower()
        mapeamento[(nome, uf_full)] = municipio['id']
    return mapeamento

def obter_codigo_ibge_eficiente(estacao: str, uf: str, mapeamento_ibge: dict) -> int:
    """
    Obtém o código IBGE de forma rápida usando o dicionário de mapeamento.
    Caso não encontre no mapeamento, tenta buscar no dicionário de ids não mapeados.
    """
    nome_preparado = preparar_nome_municipio(estacao)
    uf_nome_completo = uf_abreviatura_para_nome.get(uf.upper(), uf.upper()).lower()
    chave = (nome_preparado, uf_nome_completo)
    
    if chave in mapeamento_ibge:
        return mapeamento_ibge[chave]
    if nome_preparado in ids_nao_mapeados:
        return ids_nao_mapeados[nome_preparado]
    
    logger.warning(f"Código IBGE não encontrado para {estacao} (normalizado: {nome_preparado}, UF: {uf}).")
    return -1

# Caminhos dos arquivos Excel
caminho_arquivo = r"C:/Users/ana.brum/Desktop/DadosMeteorologicos/dados_metereologicos2.xlsx"
caminho_arquivo_final = r"C:/Users/ana.brum/Desktop/DadosMeteorologicos/dados_metereologicos3.xlsx"

# Leitura do arquivo Excel
df = pd.read_excel(caminho_arquivo)

# Em vez de extrair apenas a primeira palavra, aplica a função de preparação diretamente
# Isso garante que os nomes compostos sejam processados e as associações sejam aplicadas.
df['nome_normalizado'] = df['Estação'].apply(preparar_nome_municipio)

# Carrega os dados do IBGE e cria o dicionário de mapeamento
dados_ibge = obter_dados_ibge()
mapeamento_ibge = criar_mapeamento_ibge(dados_ibge)

# Obtém o código IBGE para cada linha de forma eficiente
df['cd_ibge'] = df.apply(lambda row: obter_codigo_ibge_eficiente(row['Estação'], row['UF'], mapeamento_ibge), axis=1)

# Remove a coluna auxiliar se não for mais necessária
df.drop(columns=['nome_normalizado'], inplace=True)

# Salva o arquivo atualizado
df.to_excel(caminho_arquivo_final, index=False)
logger.info(f"Arquivo atualizado salvo em {caminho_arquivo_final}")


KeyboardInterrupt: 